In [1]:
# Auto update of packages within the notebook
%load_ext autoreload
%autoreload 2

# Packages required for model building and analysis
import os
import sys
import numpy as np
import pandas as pd
import wikipedia

# Import custom modelling code
module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.models.run_model import *
from src.visualization.visualize import *
import src.features as features
from src.data.make_wikipedia import *

# pip install git+https://github.com/lucasdnd/Wikipedia.git --upgrade


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jocelynpender/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# Import model training data
flora_data_frame = pd.read_csv("../data/processed/flora_data_frame_full.csv", index_col=0)
train_indices = list(range(0, flora_data_frame.shape[0]))

In [6]:
# Import of Wikipedia dataset
wiki = pd.read_csv("../data/processed/combined.csv", index_col=None)
test_indices = list(range(flora_data_frame.shape[0] + 1, flora_data_frame.shape[0] + wiki.shape[0]))
flora_data_frame = pd.concat([flora_data_frame, wiki], ignore_index=True)

# Get rid of key classification
flora_data_frame.classification[flora_data_frame.classification == "key"] = "morphology"


/Users/jocelynpender/miniconda3/envs/flora-prepper/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  after removing the cwd from sys.path.


,Unnamed: 0,classification,dataset_name,length,row,row_id,species,text
0,NaN,habitat,bc,267.0,NaN,5721.0,Festuca saximontana var. saximontana,"Mesic to dry meadows, grasslands, rocky slopes..."
1,NaN,habitat,fna,78.0,NaN,9819.0,NaN,"Rock, outcrops, calcareous and volcanic boulde..."
2,NaN,morphology,bc,27.0,NaN,2305.0,Epilobium glaberrimum,Blooming Period Mid Summer
3,NaN,taxon_identification,fna,251.0,NaN,6636.0,NaN,asteraceae family Martinov unknown senecioneae...
4,NaN,habitat,bc,194.0,NaN,7239.0,Aquilegia formosa,"Mesic to moist meadows, rocky slopes, thickets..."
...,...,...,...,...,...,...,...,...
59339,56.0,Varieties,wikipedia,NaN,NaN,NaN,Astragalus purshii,There are many varieties of Astragalus purshii...
59340,57.0,Cultivation,wikipedia,NaN,NaN,NaN,Astragalus purshii,Astragalus purshii is cultivated as an ornamen...
59341,58.0,References,wikipedia,NaN,NaN,NaN,Astragalus purshii,NaN
59342,59.0,External links,wikipedia,NaN,NaN,NaN,Astragalus purshii,Jepson Manual Treatment — Astragalus purshii\n...


In [14]:
# Customize stop words for model
tokenized_stop_words = features.prepare_stop_words(custom_stop_words=["unknown", "accepted", "synonym",
                                                             "basionym", "source",
                                                             "note", "notes", "morphology", "fna_id"])
# Build DTM
custom_vec, dtm_text_counts = build_dtm_text_counts(features.flora_tokenizer, tokenized_stop_words, flora_data_frame)
dtm_text_counts.toarray()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jocelynpender/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [15]:
# Prepare data for the model
X_train = dtm_text_counts[train_indices]
y_train = flora_data_frame.iloc[train_indices].classification
X_test = dtm_text_counts[test_indices]
y_test = flora_data_frame.iloc[test_indices].classification

IndexError: index (59343) out of range

In [ ]:
clf = MultinomialNB().fit(X_train, y_train)
predicted = clf.predict(X_test)
dtm_y_test_df = pd.DataFrame(y_test).reset_index()
dtm_predictions_series = pd.Series(predicted)
results = pd.concat([dtm_y_test_df, dtm_predictions_series], axis=1)
results.rename(columns={0: 'predictions'}, inplace=True)
results = results.set_index('index')
results_flora_data_frame = pd.concat([results, flora_data_frame], axis=1, join='inner')
results_flora_data_frame